In [1]:
import json, os

In [2]:
from src.nodes.extrect_correct_field_names_nodes import get_first_few_elements, get_element_meaning, preprocess_field_info, generate_field_name_description

USER_AGENT environment variable not set, consider setting it to identify your requests.


Initializing retriever tool...
Retriever created successfully
Retriever tool created successfully
ChatOpenAI model initialized
ChatOpenAI small model initialized


In [3]:
input = {
    'file_name': 'smaller_data.json',
    'user_id' : 'p007',
    'user_session_id' : '1',
    'data_info_from_user' : "json array of events triggered by all minecraft players in a server in choronological order",
    'messages' : []
}

In [4]:
input = get_first_few_elements(input)

Checking/creating directory: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend/temp/p007/1
Opening file: smaller_data.json
First few elements extracted successfully


In [5]:
input = get_element_meaning(input)

Generating meaning of elements in data...
Meaning of elements generated successfully


In [6]:
input = preprocess_field_info(input)

Processing field information in: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend/temp/p007/1
Opening file: smaller_data.json
File data loaded successfully
Unique fields lenth: 52
New field added: event
New field added: timestamp
New field added: time
New field added: distinct_id
New field added: $city
New field added: $insert_id
New field added: $ip
New field added: $mp_api_endpoint
New field added: $mp_api_timestamp_ms
New field added: $region
New field added: mp_country_code
New field added: mp_lib
New field added: mp_processing_time_ms
New field added: server
New field added: $identified_id
New field added: $import
New field added: $lib_version
New field added: $source
New field added: $user_id
New field added: event_original_name
New field added: id
New field added: type
New field added: username
New field added: command
New field added: full-command
New field added: device
New field added: npc
New field added: world
New field added: message
New field added: to
N

In [7]:
input = generate_field_name_description(input)

Generating field description...
Generating description for field: event
New name generated : event_type
Description generated : Indicates the specific type of event triggered by players in the game. It can represent player actions like joining, commands used, NPC interactions, or chat events.
Generating description for field: timestamp
New name generated : event_timestamp
Description generated : Records the exact date and time when a specific event occurred in the game. It provides a chronological context for the actions taken by players.
Generating description for field: time
New name generated : event_time_epoch
Description generated : Represents the timestamp of the event in epoch format. This field captures the precise moment an event occurred on the server.
Generating description for field: distinct_id
New name generated : unique_player_identifier
Description generated : A unique identifier representing a player's identity across different events. This ID remains consistent for a 

In [8]:
from langgraph.graph import StateGraph, START, END

In [9]:
from src.states.extrect_correct_field_names_states import ExtractCorrectFieldNamesStates

In [16]:
from src.nodes.extrect_correct_field_names_nodes import  retrive_node

In [ ]:
def should_retrive_for_element_info(ExtractCorrectFieldNamesStates: ExtractCorrectFieldNamesStates):
    if ExtractCorrectFieldNamesStates["messages"][-1]['tool_calls']:
        return "retrieve"
    else:
        return "preprocess_field_info"

In [ ]:
workflow = StateGraph(ExtractCorrectFieldNamesStates)

workflow.add_node("get_first_few_elements", get_first_few_elements)
workflow.add_node("get_element_meaning", get_element_meaning)
workflow.add_node("preprocess_field_info", preprocess_field_info)
workflow.add_node("generate_field_name_description", generate_field_name_description)
workflow.add_node("retireve", retrive_node)

workflow.add_conditional_edges(
    "get_element_meaning",
    should_retrive_for_element_info,
    {"preprocess_field_info" : "preprocess_field_info", "retireve" : "retireve"}
)

workflow.add_edge("preprocess_field_info", "generate_field_name_description")
workflow.add_edge("generate_field_name_description", END)

workflow.add_edge(START, get_first_few_elements)


In [37]:
compiled = workflow.compile()

ValueError: Node `get_first_few_elements` is not reachable